In [16]:
# Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [17]:
contour_path = "/mnt/cloudNAS3/Adubois/Classes/ECE661/HW6/Output_Images/Contour_Extraction/"
rgb_path = "/mnt/cloudNAS3/Adubois/Classes/ECE661/HW6/Output_Images/RGB_Otsus/"
texture_path = "/mnt/cloudNAS3/Adubois/Classes/ECE661/HW6/Output_Images/Texture_Segmentation/"
histogram_paths = "/mnt/cloudNAS3/Adubois/Classes/ECE661/HW6/Histograms/"

In [18]:
# Split images into RBG:
def get_bgr_split_images(img_path):
    img = cv2.imread(img_path)
    return cv2.split(img)

In [19]:
# Display image Object:
def display_img_object(img, isOpenCV=True):
    if isOpenCV:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [20]:
# RBG Segmentation
def get_Otsus_segmentation(img_channel, img_name, iter, subfolder):
    num_bins = int(np.max(img_channel) - np.min(img_channel) + 1)
    hist, bin_edges = np.histogram(img_channel, bins=num_bins)
    
    # Get histogram probabilities
    p_i = hist / sum(hist)
    
    # List to save between class variances
    variances = np.zeros((len(bin_edges)))
    
    # Go through all threshold ts and find the one that maximizes the fisher discriminant
    # I only work with the pixels left from the previous iteration of Otsu's algorithm.
    # So, my loop for the t values can only get larger than the previous one
    for t in range(len(bin_edges)):
        # Probability of picking points from each class = sum(p(i))
        w0 = np.sum(p_i[:t])
        w1 = np.sum(p_i[t:])
        
        if w0 == 0 or w1 == 0:
            continue
        
        # Mean levels for each class = sum(i * p(i))
        i = np.arange(len(p_i))
        mu0 = np.sum(i[:t] * p_i[:t])/w0
        mu1 = np.sum(i[t:] * p_i[t:])/w1
        
        # Between class varianced
        omegaB = w0 * w1 * (mu1 - mu0)**2
        
        # Append fisher's discriminant to the variances list
        variances[t] = omegaB
        
    optimal_threshold = np.argmax(variances)
        
    # Save a graph of the histogram with the variances at each t plotted on top
    plt.figure(figsize=(10, 6))
    plt.bar(bin_edges[:-1], hist, width=np.diff(bin_edges), edgecolor="black", align="edge", label="Pixel Value Histogram")
    plt.plot(variances, color='r', label="Between class variances at each t")
    plt.axvline(x=optimal_threshold, color="k", linestyle="--", linewidth=2, label="Optimal Threshold")
    plt.legend(loc='upper right')
    plt.savefig(histogram_paths + subfolder + img_name + "_hist_" + str(iter) + ".jpg", format="jpg")
    plt.close()
    
    # Return the t that maximized fisher's ratio
    return optimal_threshold

In [21]:
# Texture Segmentation
def get_texture_ostu(grey_img, N):
    # Create canvas for output image
    output_channel = np.zeros_like(grey_img)
    
    # Pad the image with extra 0s as necessary
    padded_img = np.pad(grey_img, pad_width=N, mode="constant", constant_values=0)
    
    # Create an N by N window of the nearby pixels with 0 padding
    # Instead of using a double for loop which is slow, I do this through numpy vectorized functions
    # This will return a numpy array of shape: (H, W, 2N + 1, 2N + 1) (list of windows)
    # We do 2N + 1 since we want N on each direction from the input pixel
    window_shape = (2*N + 1, 2*N + 1)
    windows = np.lib.stride_tricks.sliding_window_view(padded_img, window_shape)
    
    # Substract the window's mean value
    window_means = np.mean(windows, axis=(-2, -1))
    windows = windows - window_means[:,:, None, None]
    
    # Compute the variance within each window
    window_variances = np.var(windows, axis=(-2, -1))
    
    # Normalize the variance to 0-1 range and scale to 255 for BW image
    output_channel = (window_variances - window_variances.min()) / (window_variances.max() - window_variances.min()) * 255

    # Return a new array with the within pixel variance as the center pixel value
    # These values are normalized to 0-255
    return output_channel.astype(np.uint8)

In [22]:
def get_contours(img):
    window_shape = (3,3)
    windows = np.lib.stride_tricks.sliding_window_view(img, window_shape)
    
    # We only want to look at windows where the center pixel is 1:
    center_pixels = windows[:, :, 1, 1]
    center_pixel_mask = center_pixels == 1
    
    # Create a mask that will look in each window and check for a 0 pixel
    # If there is a 0, it will return a 1, otherwise return a 0
    zero_mask = np.any(windows == 0, axis=(2, 3)).astype(int)
    
    final_mask = np.logical_and(center_pixel_mask, zero_mask)

    # Convert to grey scale (0->255)
    return final_mask * 255

In [30]:
def run_opening(contour, kernel_size):
    # Opening is erosion followed by dillation
    # Here we define the kernel for the area we want to run these algorithms on
    kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)
    
    # First we run erosion on the contour
    contour = cv2.erode(contour.astype(np.float32), kernel, iterations=1)
    
    # Then we run dillation:
    contour = cv2.dilate(contour.astype(np.float32), kernel, iterations=2) 
    return contour
    
def run_closing(contour, kernel_size):
    # Closing is dillation followed by erosion
    kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)

    contour = cv2.dilate(contour.astype(np.float32), kernel, iterations=3)
    contour = cv2.erode(contour.astype(np.float32), kernel, iterations=1)
    return contour
    

Calling code:

In [24]:
def run_Otsus_algorithm_iterative(iters, save_path, invert, img_path=None, img=None, img_name="", histogram_folder=""):
    """
        Runs an iterative version of Otsus algorithm. Will print the following results:
            - Seperate masks for each channel dimension  
            - A joined mask highlighting the RGB extractions for each mask  
            - The histogram with variance values at each tested threshold for Otsu's algorithm.  

    Args:
        iters (int): number of iters to run Otsus
        save_path (string): path pointing to where you want the images to be saved
        img_path (string, optional): Path to the image. Defaults to None.
        img (OpenCV Image Object, optional): OpenCV BGR Image. Defaults to None.
        img_name (str, optional): NameID for image printouts. Defaults to "".
    """

    print("Working on the following image:", img_name)
    if img_path != None:
        channels = get_bgr_split_images(img_path)
    else:
        channels = cv2.split(img)
    
    final_rgb_img = np.zeros((channels[0].shape[0], channels[0].shape[1], 3), dtype=np.uint8)
    final_mask = np.ones_like(channels[0], dtype=np.uint8)
    
    for i, channel in enumerate(channels):
        flattened_channel = channel.flatten()
        # Channel I am working on:
        channel_name = ["_blue", "_green", "_red"][i]
        print("Working on the following channel: ", channel_name)
        
        for iter in range(iters[i]):
            # As I run Otsu's algorithm, more of my pixels become 0 due to the masking.
            optimal_threshold = get_Otsus_segmentation(flattened_channel, img_name, iter, histogram_folder)
            
            # Print the optimal threshold:
            print("Optimal Threshold: ", optimal_threshold)
            
            # If the values in the channel are lessthan the threshold -> replace it with 0 in the bw_img or do the opposite.
            # This choice can be made depending on whether the foreground is a bright object, or a dark one.
            # If foreground is closer to white, you want to keep Class 1 (brighter pixels), else keep class 0 (darker pixels)
            if not invert:
                bw_img = np.zeros(channel.shape, dtype=np.uint8)
                bw_img[channel<optimal_threshold] = 255
            else:
                bw_img = np.ones_like(channel) * 255
                bw_img[channel<optimal_threshold] = 0
            
            # Fill in the channels for the final image:
            final_rgb_img[:, :, i] = bw_img
            
            # Save the masked image to a file
            cv2.imwrite(save_path + "RGB_Seperate/Iter" + str(iter) + "/" + img_name + channel_name + ".jpg", bw_img)
            
            # Update channel to new image for next iteration:
            # Use the bw_img as a mask on top of the old channel
            # channel = channel * (bw_img // 255)
            flattened_channel = flattened_channel[flattened_channel < optimal_threshold]
        
        
        # Final mask is a logical and of the masks from all of the final masks after all iterations of thresholding
        final_mask = np.logical_and(bw_img // 255, final_mask).astype(np.uint8)
            
    # Save the image with the combined masks:
    final_mask_img = final_mask * 255
    cv2.imwrite(save_path + "Final_Images/" + img_name + ".jpg", final_mask_img)
    
    # Save the final image in RBG with all channels masked seperately:
    cv2.imwrite(save_path + "RGB_Final_Masks/" + img_name + ".jpg", final_rgb_img)

In [25]:
input_folder = "/mnt/cloudNAS3/Adubois/Classes/ECE661/HW6/pics/"
img_paths = [input_folder+img_name for img_name in os.listdir(input_folder)]

In [20]:
# Otsu's Algorithm for RGB images:
# Images are in the following order: Climb, Flower, dog_small, RVL
iters = [[3,3,4], 
         [1,1,1],
         [3,3,3],
         [2,2,2]]
invert = [False, True, False, False]
for i, img_path in enumerate(img_paths):
    img_name = img_path.split("/")[-1][:-4]
    run_Otsus_algorithm_iterative(iters=iters[i], save_path=rgb_path, invert=invert[i], img_path=img_path, img=None, img_name=img_name, histogram_folder="RGB/")    
    

Working on the following image: climg
Working on the following channel:  _blue
Optimal Threshold:  103
Optimal Threshold:  64
Optimal Threshold:  37
Working on the following channel:  _green
Optimal Threshold:  122
Optimal Threshold:  77
Optimal Threshold:  49
Working on the following channel:  _red
Optimal Threshold:  134
Optimal Threshold:  86
Optimal Threshold:  62
Optimal Threshold:  40
Working on the following image: flower_small
Working on the following channel:  _blue
Optimal Threshold:  126
Working on the following channel:  _green
Optimal Threshold:  136
Working on the following channel:  _red
Optimal Threshold:  140
Working on the following image: dog_small
Working on the following channel:  _blue
Optimal Threshold:  84
Optimal Threshold:  39
Optimal Threshold:  17
Working on the following channel:  _green
Optimal Threshold:  132
Optimal Threshold:  68
Optimal Threshold:  29
Working on the following channel:  _red
Optimal Threshold:  120
Optimal Threshold:  57
Optimal Thresho

In [17]:
# Images are in the following order: Climb, Flower, dog_small, RVL
iters = [[1,1,1],
         [2,2,2],
         [1,1,1],
         [3,2,2]]
invert = [True, True, True, True]
window_sizes = [[3, 5, 7],
                [3, 5, 7],
                [5, 4, 3],
                [1, 2, 3]]

# Otsu's algorithm based on texture:
for i, img_path in enumerate(img_paths):
    img_name = img_path.split("/")[-1][:-4]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    final_mask = np.ones_like(img, dtype=np.uint8)
    
    # Initialize image to fill in with texture data
    new_img = np.zeros((img.shape[0], img.shape[1], 3))
    
    for channel_idx ,N in enumerate(window_sizes[i]):
        # Run Texture Otsu on the image:
        channel = get_texture_ostu(img, N)

        # Input the texture Otsu outputs as channels for BGR Otsu
        new_img[:,:,channel_idx] = channel
    # I now have a new image that I need to segment using BGR Otsu's Algorithm
    run_Otsus_algorithm_iterative(iters=iters[i], save_path=texture_path, invert=invert[i], img_path=None, img=new_img, img_name=img_name, histogram_folder="Texture/")    
    

Working on the following image: climg
Working on the following channel:  _blue
Optimal Threshold:  30
Working on the following channel:  _green
Optimal Threshold:  32
Working on the following channel:  _red
Optimal Threshold:  33
Working on the following image: flower_small
Working on the following channel:  _blue
Optimal Threshold:  31
Optimal Threshold:  9
Working on the following channel:  _green
Optimal Threshold:  31
Optimal Threshold:  9
Working on the following channel:  _red
Optimal Threshold:  32
Optimal Threshold:  10
Working on the following image: dog_small
Working on the following channel:  _blue
Optimal Threshold:  34
Working on the following channel:  _green
Optimal Threshold:  33
Working on the following channel:  _red
Optimal Threshold:  32
Working on the following image: rvl
Working on the following channel:  _blue
Optimal Threshold:  51
Optimal Threshold:  14
Optimal Threshold:  4
Working on the following channel:  _green
Optimal Threshold:  46
Optimal Threshold:  14

In [31]:
final_image_dirs = ["/mnt/cloudNAS3/Adubois/Classes/ECE661/HW6/Output_Images/Texture_Segmentation/Final_Images/",
                   "/mnt/cloudNAS3/Adubois/Classes/ECE661/HW6/Output_Images/RGB_Otsus/Final_Images/"]
types = ["texture", "rgb"]
for type, final_img_dir in zip(types, final_image_dirs):
    for img_name in os.listdir(final_img_dir):
        # Run the contouring on that image
        
        # Open the image:
        img = cv2.imread(final_img_dir + img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Get the image contour:
        contour = get_contours(img)
        
        # Save the images:
        cv2.imwrite(contour_path+img_name[:-4] + "_" + type + ".jpg", contour)
        
        # Run Closing on the images:
        closed_contour = run_closing(contour, 3)
        cv2.imwrite(contour_path+"Post_Closing/" + img_name[:-4] + "_" + type + ".jpg", closed_contour)
        
        # Run Opening on the images:
        opened_contour = run_opening(contour, 3)
        cv2.imwrite(contour_path+"Post_Opening/" + img_name[:-4] + "_" + type + ".jpg", closed_contour)
        